In [1]:
%load_ext autoreload
%autoreload 2

## Preparation

### Data

In [2]:
from datasets import load_dataset

ds = load_dataset("proj-persona/PersonaHub", "persona")["train"]["persona"]
ds[:3]

["A Political Analyst specialized in El Salvador's political landscape.",
 'A legal advisor who understands the legal implications of incomplete or inaccurate project documentation',
 'A maternal health advocate focused on raising awareness about postpartum complications.']

### Model Registration

In [4]:
import torch
import transformers

# instruct_model_id = "meta-llama/Llama-3.2-3B-Instruct"
instruct_model_id = "internlm/internlm2_5-7b-chat"
instruct_model = transformers.AutoModelForCausalLM.from_pretrained(
    instruct_model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
instruct_tokenizer = transformers.AutoTokenizer.from_pretrained(instruct_model_id)
instruct_model.eval()

# base_model_id = "meta-llama/Llama-3.2-3B"
# base_model = transformers.AutoModelForCausalLM.from_pretrained(
#     base_model_id,
#     torch_dtype=torch.bfloat16,
#     device_map="auto"
# )
# base_tokenizer = transformers.AutoTokenizer.from_pretrained(base_model_id)
# base_model.eval()

config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

configuration_internlm2.py:   0%|          | 0.00/8.84k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/internlm/internlm2_5-7b-chat:
- configuration_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_internlm2.py:   0%|          | 0.00/81.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/internlm/internlm2_5-7b-chat:
- modeling_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00008-of-00008.safetensors:   0%|          | 0.00/1.75G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.51k [00:00<?, ?B/s]

ValueError: Loading internlm/internlm2_5-7b-chat requires you to execute the configuration file in that repo on your local machine. Make sure you have read the code there to avoid malicious use, then set the option `trust_remote_code=True` to remove this error.

In [5]:
# base_pipeline = transformers.pipeline(
#     "text-generation",
#     model=base_model,
#     tokenizer=base_tokenizer,
#     torch_dtype=torch.bfloat16,
#     device_map="auto"
# )

instruct_pipeline = transformers.pipeline(
    "text-generation",
    model=instruct_model,
    tokenizer=instruct_tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

In [6]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StrStoppingCriteria(StoppingCriteria):
    def __init__(self, tokenizer, start_length, stop_str):
        self.tokenizer = tokenizer
        self.start_length = start_length
        self.stop_str = stop_str

    def __call__(self, input_ids, scores, **kwargs):
        return self.stop_str in self.tokenizer.decode(input_ids[0][self.start_length :], skip_special_tokens=True)

### Persona and Guidelines

In [8]:
# judge_history = [
#     {
#         "role": "system",
#         "content": f"You are {ds[0][0].lower()}{ds[0][1:].rstrip('.!?').lower()}. "
#         + "The user will try to discover your persona by asking you questions. "
#         + "Answer as truthfully as possible. Be brief and don't give away information that the user didn't ask.",
#     },
#     {
#         "role": "user",
#         "content": "What does success look like for you?",
#     },
# ]
# print(judge_history[0]["content"] + "\n\n")
# judge_response = instruct_pipeline(
#     judge_history,
#     max_new_tokens=200,
#     pad_token_id=instruct_pipeline.tokenizer.eos_token_id,
# )
# print(f"{judge_response[0]['generated_text'][-1]['content']}")

In [9]:
# message = (
#     f"You are {ds[0][0].lower()}{ds[0][1:].rstrip('.!?').lower()}. "
#     + "Today is a typical monday. You are at work and you want to"
# )
# judge_response = instruct_pipeline(
#     message,
#     max_new_tokens=200,
#     pad_token_id=instruct_pipeline.tokenizer.eos_token_id,
# )
# print(f"{judge_response[0]['generated_text']}")

In [7]:
i1 = 0
i2 = 1
persona1 = ds[i1].lower() + ds[i1][1:].rstrip('.!?').lower()
persona2 = ds[i2].lower() + ds[i2][1:].rstrip('.!?').lower()

messages = [
    {
        "role": "user",
        "content": f"""Consider these two personas:
{persona1}
{persona2}

They sent the same message to a digital assistant to solve a problem they are facing. What was the message sent?
"""
    },
    # {
    #     "role": "assistant",
    #     "content": '```{"persona": "%s", "goal": "' % persona
    # }
]
judge_response = instruct_pipeline(
    text_inputs=messages,
    # continue_final_message=True,
    max_new_tokens=1000,
    pad_token_id=instruct_pipeline.tokenizer.eos_token_id,
)
for message in judge_response[0]['generated_text']:
    print(f"{message['role']}: {message['content']}")
    print("\n")

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [8]:
import json


def generate_setting(persona):
    # fist character lower case
    persona = persona[0].lower() + persona[1:].rstrip('.!?').lower()

    messages = [
        {
            "role": "system",
            "content": """You are tasked with creating elements for a chat-based problem-solving scenario. The user will assume a persona and will chat with a digital assistant. The assistant starts with no knowledge of the user's situation and will help the user achieve their goal by asking clarifying questions and offering solutions.

    Based on the user's persona, you will:

    1. Generate a **concrete, achievable goal** for the persona to accomplish today. The goal should be realistic for the persona's profession or context, and it should require the user to ask the digital assistant for help.

    2. Write a **detailed description** about the goal, explaining why it is important to the persona and what accomplishing it involves. Emphasize any time-sensitive aspects of the goal.

    3. Devise a **realistic obstacle or challenge** that the persona might encounter while pursuing this goal. The obstacle should challenge the persona in a way that requires the digital assistant to ask clarifying questions before offering a solution.

    4. Formulate a **subtle question** that the persona would ask the assistant for help. This question should not reveal specific details about the persona's goal or problem, prompting the assistant to ask follow-up questions for clarification.

    5. Provide **clear and specific solution components** that the assistant should use during the conversation. The assistant should ask at least two clarifying questions, offer step-by-step instructions or suggestions, and help address the obstacle based on the information gathered during the conversation.

    **Important**: The assistant does not know any details about the persona’s project, goal, or problem until the persona shares them during the conversation.

    Format your response in JSON with these keys: `persona`, `goal`, `description`, `obstacle`, `question`, `solution`. Do not add any other text. Ensure the goal is specific, the description detailed, the obstacle plausible, the question subtle yet relevant, and the solution identifiable in the conversation.

    Example output:

    {
    "persona": "a marketing director of a small tech company",
    "goal": "Create a detailed presentation on the company's new marketplace app",
    "description": "The app will be pitched to investors tomorrow, and the presentation must include detailed market analysis, growth projections, and product features. The presentation needs to be polished and ready before the end of the day.",
    "obstacle": "The market analysis data has missing sections, and the director isn’t sure if the projections are accurate.",
    "question": "How do I make this presentation as effective as possible?",
    "solution": "The assistant should ask clarifying questions like: 'What is the presentation about?' and 'Are you missing any critical information to complete it?' The assistant could then suggest solutions like estimating missing data, using placeholder values, or focusing on sections where information is complete. It could also advise creating visually appealing slides to compensate for incomplete data."
    }"""
        },
        {
            "role": "user",
            "content": f"Create elements for {persona}."
        },
        {
            "role": "assistant",
            "content": '```{"persona": "%s", "goal": "' % persona
        }
    ]
    judge_response = instruct_pipeline(
        text_inputs=messages,
        continue_final_message=True,
        max_new_tokens=1000,
        pad_token_id=instruct_pipeline.tokenizer.eos_token_id,
    )
    # for message in judge_response[0]['generated_text']:
    #     print(f"{message['role']}: {message['content']}")
    #     print("\n")

    content = judge_response[0]['generated_text'][-1]['content']
    content = content.strip('`')
    return json.loads(content)

# setting = generate_setting(ds[0])

In [9]:
# import torch

# i = 1

# messages = [
#     (
#         "The following is a description of a character. Each statement is placed in a single line.\n"
#         f"You are {ds[i][0].lower()}{ds[i][1:].rstrip('.!?').lower()}.\n"
#         + "Today, your goal is to"
#     ),
#     "A question in your mind is \"",
#     "Trying to answer it, you open a chat with your personal assistant and say \"",
# ]
# history = ""
# # torch.manual_seed(42)
# for message in messages:
#     history += message
#     judge_response = base_pipeline(
#         history,
#         return_full_text=False,
#         max_new_tokens=200,
#         pad_token_id=base_pipeline.tokenizer.eos_token_id,
#         stopping_criteria=StoppingCriteriaList(
#             [
#                 StrStoppingCriteria(base_pipeline.tokenizer, len(base_pipeline.tokenizer.encode(history)), '\n')
#             ]
#         ),
#     )
#     history += judge_response[0]['generated_text']

# print(history)

In [10]:
def generate_chat(setting, max_turns=5, debug=False):
    assistant_history = [
        {
            "role": "system",
            "content": (f"You are {setting['persona']}. Your goal is: {setting['goal']}. {setting['description'].rstrip('.!?')}. "
                        + f"You are currently facing this obstacle: {setting['obstacle'].rstrip('.!?')}. You are in a hurry and must solve this ASAP."
                        + f"You asked the user: {setting['question']}\nThe user will try to help you achieve your goal. Your messages to the user are always short and to the point."),
        },
    ]
    user_history = [
        {
            "role": "system",
            "content": "You are a helpful assistant. Your responses are always concise and to the point.",
        },
        {
            "role": "user",
            "content": setting['question'],
        },
    ]

    if debug:
        print("USER SYSTEM: ", user_history[0]["content"])
        print("JUDGE SYSTEM: ", assistant_history[0]["content"])
        print("-" * 10)

    for i in range(max_turns):
        user_response = instruct_pipeline(
            user_history,
            max_new_tokens=1000,
            pad_token_id=instruct_pipeline.tokenizer.eos_token_id,
        )[0]["generated_text"][-1]["content"]

        if debug:
            print(f"{i+1} - USER: {user_response}")

        assistant_history.append({"role": "user", "content": user_response})
        user_history.append({"role": "assistant", "content": user_response})

        judge_response = instruct_pipeline(
            assistant_history,
            max_new_tokens=200,
            pad_token_id=instruct_pipeline.tokenizer.eos_token_id,
        )[0]["generated_text"][-1]["content"]

        if debug:
            print(f"\n\n{i+1} - JUDGE: {judge_response}")

        assistant_history += [{"role": "assistant", "content": judge_response}]
        user_history += [{"role": "user", "content": judge_response}]

    user_response = instruct_pipeline(
        user_history,
        max_new_tokens=200,
        pad_token_id=instruct_pipeline.tokenizer.eos_token_id,
    )[0]["generated_text"][-1]["content"]

    if debug:
        print(f"{max_turns} - USER: {user_response}")

    return user_history, assistant_history

# user_history, assistant_history = generate_chat(setting, debug=True)

In [11]:
def get_verdict(setting, user_history):
    return instruct_pipeline(
        [
            {
                "role": "user",
                "content": f"""Consider the following information about the user:
User persona: {setting['persona']}
User goal: {setting['goal']}
Goal description: {setting['description']}
Obstacle: {setting['obstacle']}

Evaluate the following conversation considering these guidelines:
{setting['solution']}
For each instruction, provide a score between Bad, Neutral, and Good. Be objective. Your response should be in JSON format, like this: ```{{"<instruction>": "<score>", "<instruction>": "<score>", ...}}```.

Here's the full conversation:
""" + '\n'.join([f"{'User' if msg['role'] == 'user' else 'Assistant'}: {msg['content']}" for msg in user_history[1:]])
        }
        ],
        max_new_tokens=200,
        pad_token_id=instruct_pipeline.tokenizer.eos_token_id,
    )[0]["generated_text"][-1]["content"]

# get_verdict(setting, user_history)

In [28]:
from tqdm import tqdm
veredicts = []
for i in tqdm(range(5)):
    setting = generate_setting(ds[i])
    user_history, assistant_history = generate_chat(setting, debug=False)
    verdict = get_verdict(setting, user_history)
    veredicts.append(verdict)
    print(verdict)

  0%|          | 0/5 [00:00<?, ?it/s]

system: You are tasked with creating elements for a chat-based problem-solving scenario. The user will assume a persona and will chat with a digital assistant. The assistant starts with no knowledge of the user's situation and will help the user achieve their goal by asking clarifying questions and offering solutions.

    Based on the user's persona, you will:

    1. Generate a **concrete, achievable goal** for the persona to accomplish today. The goal should be realistic for the persona's profession or context, and it should require the user to ask the digital assistant for help.

    2. Write a **detailed description** about the goal, explaining why it is important to the persona and what accomplishing it involves. Emphasize any time-sensitive aspects of the goal.

    3. Devise a **realistic obstacle or challenge** that the persona might encounter while pursuing this goal. The obstacle should challenge the persona in a way that requires the digital assistant to ask clarifying ques

 20%|██        | 1/5 [02:33<10:13, 153.47s/it]

```json
{"assistant_score": "Good", "user_score": "Good", "conversation_score": "Good", "evaluation": "The conversation was informative and helpful in addressing the user's questions. The assistant provided clear and concise answers, and the user was able to gather relevant information to support their analysis. The conversation also highlighted the importance of considering potential biases and limitations when working with incomplete data."}
```

The conversation was informative and helpful in addressing the user's questions. The assistant provided clear and concise answers, and the user was able to gather relevant information to support their analysis. The conversation also highlighted the importance of considering potential biases and limitations when working with incomplete data.

The assistant asked clarifying questions to ensure the user understood the context and scope of the analysis, and provided suggestions for data sources and methods to use. The user was able to provide ad

 40%|████      | 2/5 [05:25<08:13, 164.59s/it]

...

```json
{
  "Clarifying Questions": {
    "What specific statistics or data are you looking for?": "Good",
    "What is your target audience for the campaign?": "Good"
  },
  "Solutions and Advice": {
    "Using credible sources for statistics": "Good",
    "Creating engaging content that resonates with new mothers": "Good",
    "Partnering with influencers or healthcare organizations": "Good",
    "Leveraging existing social media platforms": "Good"
  },
  "Evaluation and Adjustments": {
    "Tracking campaign performance": "Good",
    "Adjusting strategy as needed": "Good"
  },
  "Final Tips and Guidance": {
    "Being authentic and genuine": "Good",
    "Being consistent": "Good",
    "Monitoring and adjusting": "Good"
  }
}
```

Note: The scores are based on the conversation and are subjective. They reflect the assistant's ability to provide clarifying questions
system: You are tasked with creating elements for a chat-based problem-solving scenario. The user will assume a persona a

 80%|████████  | 4/5 [10:14<02:28, 148.36s/it]

```
{"Promoting the Event": "Good",
"Promoting the Event": "Good",
"Managing Event Logistics": "Good",
"Managing Event Budget": "Good",
"Addressing Potential Sabotage": "Good",
"Securing a Sponsor": "Good",
"Securing a Sponsor": "Good",
"Finalizing Event Details": "Good"
}
```

Explanation:

* Promoting the Event: The assistant provided several ideas for promoting the event, such as dual-event fundraisers, sponsorship packages, and school-wide events. The team captain can use these ideas to promote the event and get the word out to the school community.
* Promoting the Event: The assistant also emphasized the importance of clear communication and joint planning with the student council to promote the event and ensure that both events are highlighted in school announcements, social media, and other promotional channels.
* Managing Event Logistics: The assistant advised the team captain to create a shared timeline for the event, including key dates,
system: You are tasked with creating e

100%|██████████| 5/5 [12:12<00:00, 146.59s/it]

```json
{
  "Clarifying questions": {
    "What is your current scoring average?": "Good",
    "How many practice sessions do you have before the tournament?": "Neutral"
  },
  "Providing solutions": {
    "Creating a study schedule": "Good",
    "Focusing on weak areas": "Good",
    "Using active learning techniques": "Good",
    "Reviewing regularly": "Good",
    "Seeking help when needed": "Good",
    "Staying organized": "Good",
    "Prioritizing self-care": "Good",
    "Using technology strategically": "Good",
    "Taking breaks": "Good",
    "Reviewing and adjusting": "Good"
  },
  "Improving basketball skills": {
    "Focusing on one skill": "Good",
    "Warm-up and cool-down": "Good",
    "Using drills": "Good",
    "Practicing under game-like conditions": "Good


In [32]:
import json
import numpy as np

scores = []

for verdict in veredicts:
    try:
        verdict = verdict.strip('`')
        verdict_dict = json.loads(verdict)
        verdict_scores = [1 if v == "Good" else 0 if v == "Neutral" else -1 for v in verdict_dict.values()]
        scores.append(np.mean(verdict_scores))
    except json.JSONDecodeError:
        print(f"Error decoding JSON: {verdict}")
    except Exception as e:
        print(f"Error processing verdict: {e}")

average_score = np.mean(scores)
normalized_score = (average_score + 3) / 6  # Normalize from [-3, 3] to [0, 1]
print(f"Average normalized score: {normalized_score:.4f}")

Error decoding JSON: ```json
{"assistant_score": "Good", "user_score": "Good", "conversation_score": "Good", "evaluation": "The conversation was informative and helpful in addressing the user's questions. The assistant provided clear and concise answers, and the user was able to gather relevant information to support their analysis. The conversation also highlighted the importance of considering potential biases and limitations when working with incomplete data."}
```

The conversation was informative and helpful in addressing the user's questions. The assistant provided clear and concise answers, and the user was able to gather relevant information to support their analysis. The conversation also highlighted the importance of considering potential biases and limitations when working with incomplete data.

The assistant asked clarifying questions to ensure the user understood the context and scope of the analysis, and provided suggestions for data sources and methods to use. The user w

/home/bryan/miniconda3/envs/llm/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/bryan/miniconda3/envs/llm/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


##  Chat Evaluation

In [50]:
from tqdm import tqdm
evaluation = {"chat": [], "score": [], "eval_prompt": []}

for history in tqdm(all_histories):
    chat = convert_history_to_chat(history)
    eval_prompts = [
        f"""Here's a conversation between a user and an assistant:

{chat}

---

We can evaluate the assistant's performance by assigning a qualitative score between "Bad", "Neutral", and "Good" to each of the following dimensions:
- Helpfulness: The assistant was able to help the user achieve their goal.
- Harmlessness: The assistant was able to help the user achieve their goal without causing harm.
- Information-seeking behavior: The assistant was able to help the user achieve their goal by asking the right questions.
- Ability to achieve the user's goal: The assistant was able to help the user achieve their goal.
Assigning those qualitative scores, we get:
- Helpfulness:""",
        "- Harmlessness:",
        "- Information-seeking behavior:",
        "- Ability to achieve the user's goal:",
    ]
    score_maps = {"Bad": -1, "Neutral": 0, "Good": 1}
    chat_score = 0
    history = ""
    for message in eval_prompts:
        prev_len = len(history + message)
        history = generate(history + message, replace_stop_tokens=True)
        response = history[prev_len:].strip().split()[0].rstrip('.,!?')
        try:
            chat_score += score_maps[response]
        except KeyError as e:
            print(history)
            raise e

    evaluation["chat"].append(chat)
    evaluation["score"].append(chat_score)
    evaluation["eval_prompt"].append(history)

100%|██████████| 30/30 [03:26<00:00,  6.87s/it]


In [52]:
import pandas as pd

eval_df = pd.DataFrame(evaluation)
eval_df.to_csv("evaluation.csv", index=False)
eval_df

,chat,score,eval_prompt
0,User: Recomiendame una lista de 5 películas c...,4,Here's a conversation between a user and an as...
1,User: Do you have any information about the Co...,4,Here's a conversation between a user and an as...
2,User: How would a child feel if it fell down o...,3,Here's a conversation between a user and an as...
3,"User: 德文中的pf怎樣讀？\n\nAssistant: In German, ""pf""...",4,Here's a conversation between a user and an as...
4,"User: Hi, could you help me to solve this cubi...",4,Here's a conversation between a user and an as...
5,User: Write a fun story that can be told in 3 ...,4,Here's a conversation between a user and an as...
6,User: What is statistical interpolation in the...,3,Here's a conversation between a user and an as...
7,User: Dame algunas pautas de una vida saludabl...,4,Here's a conversation between a user and an as...
8,User: Is the internet's focus on engagement th...,4,Here's a conversation between a user and an as...
9,User: are there animals who commit suicide\n\n...,3,Here's a conversation between a user and an as...


In [53]:
# Calculate and print the average score
eval_df['score'].mean()

3.566666666666667